# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** Deve ser um algoritmo de classificação.

Primeiro levando em conta o que a regressão deve ser usada para quando vamos ter diversos outputs com valores contínuos, neste exercício teremos poucos outputs, como “sim” ou “não” para se deve acontecer uma intervenção ou talvez a média da nota, que não possui muita variedade.

A Classificação é usada justamente em casos com poucos outputs, como apenas um valor do tipo boolean na resposta.  

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
student_data.head()

Student data read successfully!


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,passed
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,6,no
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,4,no
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,10,yes
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,2,yes
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,4,yes


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
v_passed = 0
v_failed = 0
v_rates = 0

# TODO: Calculate number of students
n_students = len(student_data)

# TODO: Calculate number of features
n_features = len(map(len, student_data)) -1 #menos o valor passed

# TODO: Calculate passing students
for _, student in student_data.iterrows():
    if (student['passed'] == 'yes'):
        v_passed+=1
    else:
        v_failed+=1
n_passed = v_passed

# TODO: Calculate failing students
n_failed = v_failed

# TODO: Calculate graduation rate
v_rates = float(n_passed) / float(n_students)
grad_rate = float(v_rates*100)

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split  
# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
##X_train = None
##X_test = None
##y_train = None
##y_test = None
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all,  test_size = num_test, random_state=43)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


E:\Users\brustrokes\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

Antes de listar os 3 escolhidos, vou exibir algo que foi realmente útil para essa escolha. É o mapa do scikit-learn para escolha do algoritmo correto, disponível neste link: http://scikit-learn.org/stable/tutorial/machine_learning_map/

**Support Vector Machines (SVM)**

    Um dos atributos que mais me chamou atenção nesse exercício é a quantidade de features (30) e a quantidade de alunos (395), me parece até um caso de "Curse of dimension" sendo assim necessitamos de um algoritmo que consiga trabalhar bem nesse cenário. Uma das vantagens do SVM é ele ser "Still effective in cases where number of dimensions is greater than the number of samples." isso é, bem efetivo de muitas dimensões com poucos registros.
    Outras vantagens:
        Efetivo em espaços dimensionais elevados.
        Usa um subconjunto de pontos de treinamento na função de decisão (chamados vetores de suporte), portanto também é eficiente na memória.
       Versátil: diferentes funções do Kernel podem ser especificadas para a função de decisão.
    Desvantagens:
        Se o número de características é muito maior do que o número de amostras, o método deve dar desempenhos fracos.
        As SVMs não fornecem diretamente estimativas de probabilidade, estas são calculadas usando a five-fold cross-validation, o que tem um custo alto de processamento.
        
    Exemplo real: Biomedcentral
    Referência: https://bmcmedinformdecismak.biomedcentral.com/articles/10.1186/1472-6947-10-16
    Título: Application of support vector machine modeling for prediction of common diseases: the case of diabetes and pre-diabetes
        Foi usado o SVM para classificar pessoas com e sem doenças comuns. Ilustrando o método para detectar pessoas com diabetes e pré-diabetes numa amostra representativa transversal da população dos E.U.A. Foram usados dados de 1999-2004 da National Health and Nutrition Examination Survey (NHANES) para desenvolver e validar modelos SVM para dois esquemas de classificação: Esquema de Classificação I (diabetes diagnosticada ou não diagnosticada versus pré-diabetes ou não diabetes) e Esquema de Classificação II Diabetes não diagnosticado ou pré-diabetes versus sem diabetes). Os modelos SVM foram usados para selecionar conjuntos de variáveis que renderiam a melhor classificação de indivíduos nessas categorias de diabetes.
        Para o esquema de classificação I, o conjunto de variáveis relacionadas ao diabetes com o melhor desempenho de classificação incluiu história familiar, idade, raça e etnia, peso, altura, circunferência da cintura, índice de massa corporal (IMC) e hipertensão. Para o Esquema de Classificação II, duas variáveis adicionais - sexo e atividade física - foram incluídas. As habilidades discriminativas dos modelos SVM para os Esquemas de Classificação I e II, de acordo com a área sob a curva característica de operação do receptor (ROC), foram de 83,5% e 73,2%, respectivamente. A ferramenta baseada na Web - Diabetes Classifier foi desenvolvida para demonstrar um aplicativo amigável que permite a avaliação individual ou de grupo com um limite configurável e definido pelo usuário.

        
**Logistic Regression**

    Nesse exercicio, existem muitas caracteristicas que podem estar relacionadas, como traveltime x studytime ou goout x Walc e muitas outras, nessa regrassão essas muitas variaveis são regularizadas.
    Vantagens:
    Muitas maneiras de regularizar o modelo para tolerar alguns erros e evitar excesso de ajuste
    Ao contrário de Naive Bayes, não temos de nos preocupar com características correlacionadas
    Ao contrário da SVM, podemos facilmente captar novos dados usando o método gradient descent 
    Desvantagens:
    É um modelo linear e não pode capturar qualquer relação complexa entre os dados ea variável alvo. É um modelo de alta polarização e dado dados suficientes, outros modelos mais complexos com menor viés podem superá-lo.
    
    Exemplo real: Smartdrill
    Referência: http://smartdrill.com/logistic-regression.html
    Título: Credit Risk Analysis Using Logistic Regression Modeling
     Um oficial de crédito em um banco quer ser capaz de identificar características que são indicativas de pessoas que são susceptíveis de inadimplência em empréstimos e, em seguida, usar essas características para discriminar entre bons e maus riscos de crédito.
     Este estudo de caso utiliza informações de 850 clientes passados e potenciais para executar uma Análise de Regressão Logística. Destes, 717 casos são clientes que receberam empréstimos anteriormente. Foi usado uma amostra aleatória de 513 desses 717 clientes para criar um modelo de risco. Foi deixado de lado os restantes 204 clientes como amostra de retenção ou de validação para testar o modelo de risco de crédito; Usou o modelo para classificar os 133 potenciais clientes como bons ou maus riscos de crédito.
     O modelo classificou corretamente cerca de 96% dos não-inadimplentes da amostra modeladora e cerca de 76% dos inadimplentes da amostra modeladora, para uma porcentagem global de classificação correta de cerca de 92%. Da mesma forma, quando aplicado ao holdout ou amostra de validação, o modelo identificou corretamente cerca de 95% dos não inadimplentes e cerca de 81% dos inadimplentes, para uma porcentagem global de classificação correta de cerca de 92%.
     
**K-Nearest Neighbors (KNeighbors)**

Como temos poucos dados para realizar nosso estudo, faz sentido manter tudo para fazer previsões em vez de gerar um modelo.

Vantagens:
O treinamento não tem custo. Funciona bem com dados ruidosos. Nenhuma compreensão dos dados é necessária.
Simplicidade, eficácia, intuitividade e desempenho competitivo da classificação em muitos domínios.

Desvantagens:
Precisa de memória para armazenar todos os dados. Mais lento nas previsões. Todos os recursos têm o mesmo peso.

    Exemplo real:  Int. Journal of Engineering Research and Applications 
    Referência: http://www.ijera.com/papers/Vol3_issue5/DI35605610.pdf
    Título: Application of K-Nearest Neighbor (KNN) Approach for Predicting Economic Events: Theoretical Background

    Devido aos efeitos do sofrimento financeiro das empresas sobre as partes interessadas, os modelos de previsão de desastres financeiros têm sido uma das áreas mais atraentes na pesquisa financeira. Nos últimos anos, após a crise financeira global, o número de empresas falidas aumentou. Uma vez que o sofrimento financeiro das empresas é o primeiro estágio de falência, o uso de índices financeiros para prever o sofrimento financeiro atraiu muita atenção dos acadêmicos, bem como das instituições econômicas e financeiras. Embora nos últimos anos tenham sido aumentados os estudos sobre a previsão do sofrimento financeiro das empresas no Irão, a maior parte dos esforços têm explorado métodos estatísticos tradicionais; E apenas alguns estudos têm utilizado métodos não paramétricos

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [7]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
from sklearn import svm
# from sklearn import model_B
from sklearn.linear_model import LogisticRegression
# from skearln import model_C
from sklearn import neighbors

# TODO: Initialize the three models
clf_A = svm.SVC()
clf_B = LogisticRegression(random_state=42)
clf_C = neighbors.KNeighborsClassifier()

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

train_sets = [(X_train_100, y_train_100), (X_train_200, y_train_200), (X_train_300, y_train_300)]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)
print("-----SVM-------")
for X, y in train_sets:
    train_predict(clf_A, X, y, X_test, y_test)
print("-----LogisticRegression-------")
for X, y in train_sets:
    train_predict(clf_B, X, y, X_test, y_test)
print("-----KNeighbors-------")
for X, y in train_sets:
    train_predict(clf_C, X, y, X_test, y_test)

-----SVM-------
Training a SVC using a training set size of 100. . .
Trained model in 0.0020 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.8447.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.8824.
Training a SVC using a training set size of 200. . .
Trained model in 0.0040 seconds
Made predictions in 0.0020 seconds.
F1 score for training set: 0.8844.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.8395.
Training a SVC using a training set size of 300. . .
Trained model in 0.0070 seconds
Made predictions in 0.0050 seconds.
F1 score for training set: 0.8690.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.8408.
-----LogisticRegression-------
Training a LogisticRegression using a training set size of 100. . .
Trained model in 0.0010 seconds
Made predictions in 0.0030 seconds.
F1 score for training set: 0.8725.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.7733.
Training a LogisticRegression using a training

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - SVM**  

| Training Set Size |      Training Time      | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0020          |       0.0010           |      0.8447      |     0.8824      |
| 200               |         0.0030          |       0.0030           |      0.8844      |     0.8395      |
| 300               |         0.0070          |       0.0050           |      0.8690      |     0.8408      |

** Classifer 2 - Logistic Regression**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0010          |        0.0000          |      0.8725      |     0.7733      |
| 200               |         0.0020          |        0.0000          |      0.8592      |     0.8000      |
| 300               |         0.0020          |        0.0000          |      0.8078      |     0.8054      |

** Classifer 3 - K-Nearest Neighbors**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0000          |        0.0030          |      0.8475      |     0.8456      |
| 200               |         0.0010          |        0.0030          |      0.8414      |     0.8212      |
| 300               |         0.0010          |        0.0060          |      0.8475      |     0.8456      |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

No geral tanto o modelo do SVM quanto do K-Nearest Neighbors parecem promissores.
Neste estudo acredito que o SVM é melhor qualificado, mesmo sendo levemente mais lento que o K-Nearest Neighbors, apresentou melhores resultados finais. Como a base de dados não possui muitos registros o tempo de execução(treino,testes) entre os dois modelos não será muito discrepante. No entanto se o número de dados for maior acredito que o K-Nearest Neighbors seria o melhor modelo para a solução.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

Como muitos outros algoritmos de machine learning, os SVMs precisam de alguns dados para começar com o que já está classificado (o conjunto de treinamento) e tentam prever um conjunto de dados não classificados (o conjunto de testes).
Este modelo trabalha muito bem para o cenário do nosso exercicio de estudantes. Uma vez que temos 30 atributos diferentes para apenas 395 estudantes, que é nossa base de aprendizado e testes.
Os dados que temos muitas vezes têm um monte de características diferentes, e assim podemos terminar traçando cada item de dados como um ponto no espaço, com o valor de cada característica sendo o valor em uma coordenada particular.
O que queremos fazer é encontrar alguma linha que divide os dados entre os dois grupos de dados classificados de forma diferente, assim como nós podemos. Esta será a linha de modo que as distâncias do ponto mais próximo em cada um dos dois grupos será mais distante.Uma vez que temos essa linha, o chamamos de classificador.
Você pode estender o exemplo de dois recursos para muitos outros recursos, como idade,studytime,traveltime ,etc, mas a idéia ainda permanece a mesma. Encontre a linha  que separa os dois grupos de dados o máximo possível e, em seguida, veja de que lado os novos dados apontam.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.



In [38]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn import grid_search, datasets
from sklearn.metrics import fbeta_score, make_scorer
start = time() 
# TODO: Create the parameters list you wish to tune
parameters =[{
    'C': [0.1, 1, 10, 100],
    'kernel': ['rbf'],
    'gamma': [0.01, 0.1, 1, 10]
 }]
# TODO: Initialize the classifier
clf = svm.SVC()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = grid_search.GridSearchCV(clf, parameters, f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_
end = time()
# Report the final F1 score for training and testing after parameter tuning
print "Made predictions in {:.4f} seconds.".format(end - start)

print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.7070 seconds.
Made predictions in 0.0060 seconds.
Tuned model has a training F1 score of 0.9794.
Made predictions in 0.0020 seconds.
Tuned model has a testing F1 score of 0.8466.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **
O F1 Score após "tunado" conseguiu valores superiores no treino ao valor padrão obtido anteriormente. Mas não o valor de teste.
O treino chegou ao incrível valor de 0.9794, comparando com o valor sem tunning o melhor valor era de 0.8844 (treino de 200)
Já o teste ficou abaixo, com o valor de 0.8466, comparando com o valor sem tunning o melhor valor era de 0.8824 (treino de 100)

Para avaliação testei diversos kernels para o F1Score, Com os kernels ['linear', 'poly', 'rbf', 'sigmoid'].O responsável pelos valores foi o kernel rbf, por isso deixei apenas ele parametrizado, já que colocando todos os kernels,o modelo final também ficou muito mais lento, levando 26.7500 segundos, contra milésimos do modelo anterior.


> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.